# 패키지 설치



In [ ]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 603 kB/s 
     |████████████████████████████████| 2.8 MB 7.8 MB/s 
     |████████████████████████████████| 57 kB 4.5 MB/s 
     |████████████████████████████████| 398 kB 57.2 MB/s 
     |████████████████████████████████| 8.0 MB 47.9 MB/s 
     |████████████████████████████████| 806 kB 74.0 MB/s 
     |████████████████████████████████| 636 kB 56.3 MB/s 
     |████████████████████████████████| 829 kB 64.6 MB/s 
     |████████████████████████████████| 140 kB 25.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x39d40000 @  0x7f805970f615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0

# 구글 드라이브 연동하기


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정

In [ ]:
#/gdrive/My Drive/nlpbook/checkpoint-qa

In [ ]:
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="bert-base-multilingual-cased",
    downstream_model_dir="/gdrive/My Drive/캡스톤디자인/KorQuAD/fine_tuning_model/ckpt_model",
    max_seq_length=256,
    max_query_length=32,
)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


downstream_model_checkpoint_fpath: /gdrive/My Drive/캡스톤디자인/KorQuAD/fine_tuning_model/ckpt_model/epoch=0-val_loss=0.52.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [ ]:
import torch
from transformers import BertConfig, BertForQuestionAnswering
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [ ]:
import numpy as np

In [ ]:
def _get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)
  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes

def get_answer_list(start_indexes, end_indexes, question_tokens, tokens):
  answer_list = []
  for start_index in start_indexes:
    for end_index in end_indexes:
      if start_index <= len(question_tokens):
        continue
      if start_index >= len(tokens):
        continue
      if end_index >= len(tokens):
        continue
      if end_index < start_index:
        continue
      length = end_index - start_index + 1
      if length > 50:
        continue
      answer_list.append((start_index, end_index))
    return answer_list

In [ ]:
import math
def _compute_softmax(scores):
  """Compute softmax probability over raw logits."""


  max_score = None
  for score in scores:
    if max_score is None or score > max_score:
      max_score = score

  exp_scores = []
  total_sum = 0.0
  for score in scores:
    x = math.exp(score - max_score)
    exp_scores.append(x)
    total_sum += x

  probs = []
  for score in exp_scores:
    probs.append(score / total_sum)
  return probs

In [ ]:
start_pred = ''
end_pred =''
n_best_size = 20
def inference_fn(question, context):
    if question and context:
        question_tokens = '[CLS]' + question + '[SEP]'
        context = context + '[SEP]'
        question_tokens = tokenizer.tokenize(question_tokens)
        context_tokens = tokenizer.tokenize(context)
        tokens = question_tokens + context_tokens
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=512
       )
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=512,
            return_token_type_ids=True,
        )
        
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})            
            start_indexes = _get_best_indexes(outputs.start_logits[0], n_best_size)
            end_indexes = _get_best_indexes(outputs.end_logits[0], n_best_size)
            answer_list = get_answer_list(start_indexes, end_indexes, question_tokens, tokens)
            if len(answer_list) == 0:
              pred_text = '[CLS]'
            else:
              scores_index = [[start, end] for start, end in answer_list]
              
              scores = [outputs.start_logits[0][start] + outputs.end_logits[0][end] for start, end in answer_list]
              
              max_index = np.argmax(_compute_softmax(scores))
              start_pred, end_pred = scores_index[max_index]
              #start_pred = outputs.start_logits.argmax(dim=-1).item()
              #end_pred = outputs.end_logits.argmax(dim=-1).item()
              pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [ ]:
q = '인공지능의 정의는?'
c = '인공지능 또는 아티피셜 인텔리전스은 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현하려는 컴퓨터과학의 세부분야 중 하나이다. 정보공학 분야에 있어 하나의 인프라 기술이기도 하다.'

In [ ]:
tokenizer.tokenize(q)

['인', '##공', '##지', '##능', '##의', '정', '##의', '##는', '?']

In [ ]:
truncated_query = tokenizer.encode(
            q,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )

query = tokenizer.encode(
            c,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )

In [ ]:
inference_fn(q, c)

{'answer': '[CLS]',
 'context': '인공지능 또는 아티피셜 인텔리전스은 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현하려는 컴퓨터과학의 세부분야 중 하나이다. 정보공학 분야에 있어 하나의 인프라 기술이기도 하다.[SEP]',
 'question': '인공지능의 정의는?'}

##성능 측정


In [ ]:
import json

dev_path = '/gdrive/My Drive/캡스톤디자인/KorQuAD/bert-master/KorQuAD_v1.0_dev.json'

In [ ]:
with open(dev_path, 'r') as f:
  json_data = json.load(f)


In [ ]:
json_dict = {}
i = 0
for data in json_data['data']:
  print(i)
  i += 1
  for paragraph in data['paragraphs']:
    context = paragraph['context']
    for qa in paragraph['qas']:
      question = qa['question']
      question_id = qa['id']
      answer = inference_fn(question, context)
      json_dict[question_id] = answer['answer'].encode('unicode_escape').decode('utf-8')


In [ ]:
with open('/gdrive/My Drive/캡스톤디자인/KorQuAD/bert-master/pred.json', 'w') as make_file:
    json.dump(json_dict, make_file, indent="\t")